# Stats on copy number losses detected in comparison to the gold standard

## Load packages

In [1]:
import pandas as pd
from pybedtools import BedTool

## Load parsed copy number losses detected by the tools

In [2]:
columns = ['chrom', 'start', 'end']
cnvkit_table = pd.read_table('results/cnvkit.losses.bed', header=None, names=columns)
cnvpytor_table = pd.read_table('results/cnvpytor.losses.bed', header=None, names=columns)
exomedepth_table = pd.read_table('results/exomedepth.losses.bed', header=None, names=columns)
amplicnv_table = pd.read_table('results/amplicnv.losses.bed', header=None, names=columns)


## Load benchmark copy losses

In [3]:
benchmark_table = pd.read_table('results/AmpliSeqExome.20141113.intersected.giab.del.genes.grouped.bed',
                                header=None, names=columns + ['gene'])


## Compute overlap by gene for each tool output

### Function to compute benchmark copy number losses' coverage in a given bed

In [4]:
def compute_coverage(benchmark: pd.DataFrame, tool_table: pd.DataFrame):
    a = BedTool(benchmark.to_string(header=False, index=False, index_names=False), from_string=True)
    b = BedTool(tool_table.to_string(header=False, index=False, index_names=False), from_string=True)
    names = ['chrom', 'start', 'end', 'gene', 'overlaps', '# covered bases', 'length', 'fraction']
    coverage = a.coverage(b).to_dataframe(disable_auto_names=True, names=names).sort_values(['chrom', 'start'])
    return coverage.set_index(['chrom', 'start', 'end'])

### Compute benchmark coverage for each tool

In [5]:
cnvkit_cov = compute_coverage(benchmark_table, cnvkit_table)
cnvpytor_cov = compute_coverage(benchmark_table, cnvpytor_table)
exomedepth_cov = compute_coverage(benchmark_table, exomedepth_table)
amplicnv_cov = compute_coverage(benchmark_table, amplicnv_table)

In [6]:
all_tools_cov = pd.concat([
    cnvkit_cov[['gene', 'overlaps']],
    cnvpytor_cov['overlaps'],
    exomedepth_cov['overlaps'],
    amplicnv_cov['overlaps']
], axis=1)
all_tools_cov.columns = ['gene', 'cnvkit', 'cnvpytor', 'exomedepth', 'amplicnv']
print(all_tools_cov.head(len(all_tools_cov)))

                               gene  cnvkit  cnvpytor  exomedepth  amplicnv
chrom start     end                                                        
chr1  89476409  89477767       GBP3       0         0           1         0
      108735110 108735337  SLC25A24       0         1           1         1
      145096464 145096695    SEC22B       0         0           0         0
      152573151 152573551     LCE3C       0         0           0         0
      152586175 152586709     LCE3B       0         0           0         0
chr11 4967379   4968456      OR51A4       0         0           0         0
chr12 53086335  53086754      KRT77       0         0           1         0
chr5  140222137 140223359    PCDHA8       0         0           1         1
      140227897 140230614    PCDHA9       0         0           1         1
      140235620 140238199   PCDHA10       0         0           1         1
      147553752 147554778   SPINK14       0         1           0         0
chr7  100331

## Retain losses on autosomal chromosomes only

In [7]:
sex_chromosomes = ['chrX', 'chrY']
cnvkit_table = cnvkit_table[~cnvkit_table['chrom'].isin(sex_chromosomes)]
cnvpytor_table = cnvpytor_table[~cnvpytor_table['chrom'].isin(sex_chromosomes)]
exomedepth_table = exomedepth_table[~exomedepth_table['chrom'].isin(sex_chromosomes)]
amplicnv_table = amplicnv_table[~amplicnv_table['chrom'].isin(sex_chromosomes)]

## Count the number of detected losses by tool

In [8]:
print(f'cnvkit: {len(cnvkit_table)}')
print(f'cnvpytor: {len(cnvpytor_table)}')
print(f'exomedepth: {len(exomedepth_table)}')
print(f'amplicnv: {len(amplicnv_table)}')

cnvkit: 0
cnvpytor: 13050
exomedepth: 312
amplicnv: 395
